In [12]:
import pandas as pd
import numpy as np
from utils import why
from utils import dataset as DS

In [3]:
df = pd.read_csv('./results/overaldownload.csv')

# # Reshaping the DataFrame into the desired format
df_melted = df.melt(id_vars=["Dataset"], var_name="Method", value_name="Accuracy")

df_melted['IsEven'] = False
df_melted= df_melted[df_melted['Method']!= "ROCKET"] 

In [4]:
provision_df = pd.read_csv('./results/hydra.csv',names = ['Dataset', 'Accuracy', 'a' , 'b'])
provision_dataset = provision_df.Dataset.tolist()
# newer version
provision_dataset = df_melted.Dataset.tolist()

In [5]:
# hydra_dic = './Run/time_hydra.csv'
raster1_dic = './results/raster.csv'
df_raster1 = pd.read_csv(raster1_dic)


In [6]:
hydra_dic = "./results/hydra.csv"
inception_dic = "./results/inceptionNet.csv"
resnet_dic = "./results/resnet.csv"
rocket_dic = "./results/rocket.csv"


df_hydra = pd.read_csv(
    hydra_dic, names=["Dataset", "Method", "Accuracy", "IsEven", "Time"]
)

df_raster1 = pd.read_csv(raster1_dic)

# Inception
df_inception = pd.read_csv(inception_dic, names=["Method", "Dataset", "Accuracy"])


# resnet
df_resnet = pd.read_csv(resnet_dic, names=["Method", "Dataset", "Accuracy"])



df_rocket = pd.read_csv(rocket_dic, names=["Dataset", "Method", "Accuracy", "IsEven"])


df_concat = pd.concat([df_hydra, df_raster1, df_rocket, df_melted], ignore_index=True)
df_concat = df_concat.drop("IsEven", axis=1)
df_concat = df_concat.drop("Time", axis=1)

In [7]:
df_concat

,Dataset,Method,Accuracy
0,ACSF1,Hydra,0.840000
1,Adiac,Hydra,0.808184
2,AllGestureWiimoteX,Hydra,0.744286
3,AllGestureWiimoteY,Hydra,0.760000
4,AllGestureWiimoteZ,Hydra,0.721429
...,...,...,...
1595,Wine,STC,0.703704
1596,WordSynonyms,STC,0.659875
1597,Worms,STC,0.727273
1598,WormsTwoClass,STC,0.818182


In [8]:
df_total = df_concat.groupby(['Method','Dataset'])['Accuracy'].mean().reset_index()
df_total = df_total[df_total.Dataset.isin(provision_dataset)]
# df_total
all_methods = np.unique(df_total['Method'])
df_total = df_total[df_total['Method'].isin(['ISD','InceptionTime',
                                    'Arsenal','RASTER2','ROCKET',
                                    'TS-CHIEF','MiniROCKET'])]
df_total['Method'] = df_total['Method'].replace('RASTER2', 'RASTER')

# Total

In [9]:
df_total.groupby(['Method'])['Accuracy'].mean()

Method
Arsenal          0.854120
ISD              0.865113
InceptionTime    0.857782
MiniROCKET       0.857389
RASTER           0.851961
ROCKET           0.856959
TS-CHIEF         0.864847
Name: Accuracy, dtype: float64

In [ ]:
print( ' datasets where MiniROCKET is better than ISD')
miniwasbetter = why.compare_2method(df_total, 'MiniROCKET','ISD')
miniwasbetter.merge( DS.load_dataset_description(),on=['Dataset'])

,Dataset,MiniROCKET_Accuracy,ISD_Accuracy,Difference,Train Size,Test Size,Length,No. of Classes,Type
0,SonyAIBORobotSurface1,0.901276,0.852468,0.048808,20,601,70,2,SENSOR
1,SonyAIBORobotSurface2,0.913606,0.880378,0.033228,27,953,65,2,SENSOR
2,Mallat,0.945131,0.917982,0.027150,55,2345,1024,8,SIMULATED


In [15]:
print( ' datasets where RASTER is better than ISD')
rasterwasbetter = why.compare_2method(df_total, 'RASTER','ISD')
rasterwasbetter.merge(DS.load_dataset_description(),on=['Dataset'])

 datasets where RASTER is better than ISD


,Dataset,RASTER_Accuracy,ISD_Accuracy,Difference,Train Size,Test Size,Length,No. of Classes,Type
0,SonyAIBORobotSurface1,0.896839,0.852468,0.044370,20,601,70,2,SENSOR
1,Phoneme,0.289557,0.255802,0.033755,214,1896,1024,39,SOUND
2,DistalPhalanxTW,0.697842,0.664269,0.033573,400,139,80,6,IMAGE
3,SonyAIBORobotSurface2,0.903463,0.880378,0.023085,27,953,65,2,SENSOR


In [ ]:
print( ' datasets where RASTER is better than ISD')
rasterwasbetter = why.compare_2method(df_total, 'RASTER','ISD')
rasterwasbetter.merge(DS.load_dataset_description(),on=['Dataset'])

# Pruned version

In [17]:
df_pruned = df_total.merge(DS.load_dataset_description(),on=['Dataset'])
df_pruned['ratio'] = df_pruned['Train Size']/df_pruned['No. of Classes']
df_pruned = df_pruned[df_pruned['ratio']>4]
df_pruned = df_pruned[df_pruned['Length']>100]
df_pruned.groupby(['Method'])['Accuracy'].mean()

Method
Arsenal          0.864472
ISD              0.871918
InceptionTime    0.859624
MiniROCKET       0.862119
RASTER           0.868011
ROCKET           0.864915
TS-CHIEF         0.870228
Name: Accuracy, dtype: float64

In [18]:
mini_sig_dataset = why.compare_2method(df_pruned, 'MiniROCKET', 'RASTER',threshold=0.02)['Dataset'].tolist()
rast_sig_dataset = why.compare_2method(df_pruned, 'RASTER', 'MiniROCKET',threshold=0.02)['Dataset'].tolist()

selected_dataset = [*mini_sig_dataset, *rast_sig_dataset]
df_challenge = df_pruned[df_pruned["Dataset"].isin(selected_dataset)]
df_pruned[df_pruned["Dataset"].isin(selected_dataset)].groupby("Method")['Accuracy'].mean()

Method
Arsenal          0.805815
ISD              0.827805
InceptionTime    0.770115
MiniROCKET       0.796399
RASTER           0.818099
ROCKET           0.799136
TS-CHIEF         0.823285
Name: Accuracy, dtype: float64

In [19]:
df_pruned[df_pruned["Dataset"].isin(selected_dataset)].groupby("Method")['Accuracy'].mean()

Method
Arsenal          0.805815
ISD              0.827805
InceptionTime    0.770115
MiniROCKET       0.796399
RASTER           0.818099
ROCKET           0.799136
TS-CHIEF         0.823285
Name: Accuracy, dtype: float64